In [2]:
##### Imports #####

from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import multiprocessing
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import Sequence
from albumentations import Compose, VerticalFlip, HorizontalFlip, Rotate, GridDistortion,CenterCrop
from copy import deepcopy
import cv2
import os, glob
from sklearn.metrics import confusion_matrix, accuracy_score, cohen_kappa_score
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

In [ ]:
##### Grab the data and split into training, validation, and testing datasets #####

np.random.seed(10)
sample_size = 400

test_imgs_folder = '/home/valerie/Remote Sensing Data Analysis/Group Project/test_images/'
train_imgs_folder = '/home/valerie/Remote Sensing Data Analysis/Group Project/train_images/'
train_df = pd.read_csv('/home/valerie/Remote Sensing Data Analysis/Group Project/train.csv')

train_df = train_df[~train_df['EncodedPixels'].isnull()]
train_df['Image'] = train_df['Image_Label'].map(lambda x: x.split('_')[0])
train_df['Class'] = train_df['Image_Label'].map(lambda x: x.split('_')[1])
classes = train_df['Class'].unique()
train_df = train_df.groupby('Image')['Class'].agg(set).reset_index()
for class_name in classes:
    train_df[class_name] = train_df['Class'].map(lambda x: 1 if class_name in x else 0)
    
train_df = train_df[train_df['Class'].map(len) == 1]   #1348 images like this

train_df_v1 = train_df.sample(sample_size)
train_df_v2 = train_df.sample(sample_size)
img_2_ohe_vector = {img:vec for img, vec in zip(train_df['Image'], train_df.iloc[:, 2:].values)}


train_imgs, val_imgs = train_test_split(train_df_v1['Image'].values, 
                                        test_size=0.2, 
                                        stratify=train_df_v1['Class'].map(lambda x: str(sorted(list(x)))), # sorting present classes in lexicographical order, just to be sure
                                        random_state=43)

train_imgs, test_imgs = train_test_split(train_df_v2['Image'].values, 
                                        test_size=0.2, 
                                        stratify=train_df_v2['Class'].map(lambda x: str(sorted(list(x)))), # sorting present classes in lexicographical order, just to be sure
                                        random_state=43)

#print(train_df['Class'][4])
#print(len(train_df['Class'][4]))
#print(len(train_df['Class']))

train_df_v1.head(5)

In [ ]:
##### Create DataGenerator class to organize the data arrays #####

class DataGenerator(Sequence):
    def __init__(self, images_list=None, folder_imgs=train_imgs_folder, 
                 batch_size=1, shuffle=True, augmentation=None,
                 resized_height=224, resized_width=224, num_channels=3):
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.augmentation = augmentation
        if images_list is None:
            self.images_list = os.listdir(folder_imgs)
        else:
            self.images_list = deepcopy(images_list)
        self.folder_imgs = folder_imgs
        self.len = len(self.images_list) // self.batch_size
        self.resized_height = resized_height
        self.resized_width = resized_width
        self.num_channels = num_channels
        self.num_classes = 4
        self.is_test = not 'train' in folder_imgs
        if not shuffle and not self.is_test:
            self.labels = [img_2_ohe_vector[img] for img in self.images_list[:self.len*self.batch_size]]

    def __len__(self):
        return self.len
    
    def on_epoch_start(self):
        if self.shuffle:
            random.shuffle(self.images_list)

    def __getitem__(self, idx):
        current_batch = self.images_list[idx * self.batch_size: (idx + 1) * self.batch_size]
        X = np.empty((self.batch_size, self.resized_height, self.resized_width, self.num_channels))
        y = np.empty((self.batch_size, self.num_classes))


        for i, image_name in enumerate(current_batch):
            path = os.path.join(self.folder_imgs, image_name)
            #try:
            img = cv2.resize(cv2.imread(path), (self.resized_height, self.resized_width)).astype(np.float32)
            #  print(img.shape)
            #except:
              # if it fails, who cares there are more images
            #  img = np.zeros([self.resized_height, self.resized_width, 3], dtype=np.float32)
            #  print('error')  
              #self.__getitem__(idx+1)  


            if not self.augmentation is None:
                print(img.shape)
                augmented = self.augmentation(image=img)
                img = augmented['image']
                print(img.shape)
            X[i, :, :, :] = img#/255.0
            if not self.is_test:
                y[i, :] = img_2_ohe_vector[image_name]
        return X, y

    def get_labels(self):
        if self.shuffle:
            images_current = self.images_list[:self.len*self.batch_size]
            labels = [img_2_ohe_vector[img] for img in images_current]
        else:
            labels = self.labels
        return np.array(labels)

albumentations_train = Compose([VerticalFlip(), HorizontalFlip(), Rotate(limit=20), GridDistortion()], p=1)

data_generator_train = DataGenerator(train_imgs, augmentation=None)
data_generator_val = DataGenerator(val_imgs, augmentation=None)
data_generator_test = DataGenerator(test_imgs, augmentation=None)


#data_generator_test.__getitem__(1)[1]

In [ ]:
##### Get data arrays to work with #####

X_train = np.zeros((int(sample_size*0.8), 224, 224, 3))
Y_train = np.zeros((int(sample_size*0.8), 4))
X_val = np.zeros((int(sample_size*0.2), 224, 224, 3))
Y_val = np.zeros((int(sample_size*0.2), 4))
X_test = np.zeros((int(sample_size*0.2), 224, 224, 3))
Y_test = np.zeros((int(sample_size*0.2), 4))

for i in range(0, int(sample_size*0.8)):     #####batch size is 32
    X_train[i,:,:,:] = data_generator_train.__getitem__(i)[0]  #shape(0.8*sample_size, 224, 224, 3)
    Y_train[i,:] = data_generator_train.__getitem__(i)[1]      #shape(0.8*sample_size, 4)
    
for i in range(0, int(sample_size*0.2)):
    X_val[i,:,:,:] = data_generator_val.__getitem__(i)[0]  #shape(0.2*sample_size, 224, 224, 3)
    Y_val[i,:] = data_generator_val.__getitem__(i)[1]      #shape(0.2*sample_size, 4)
    X_test[i,:,:,:] = data_generator_test.__getitem__(i)[0]  #shape(0.2*sample_size, 224, 224, 3)
    Y_test[i,:] = data_generator_test.__getitem__(i)[1]      #shape(0.2*sample_size, 4)

In [ ]:
#Y_test
#60 // 32

#data_generator_test.__getitem__(1)[1]

In [ ]:
##### Normalize the image data #####
X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
X_test = X_test.astype('float32')
#X_train /= 255
#X_val /= 255
#X_test /= 255

In [ ]:
##### Build a simple CNN model #####

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(4))
model.add(Activation('softmax'))

In [ ]:
##### Compile the CNN model #####

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
##### Fit the CNN model #####

score_history = model.fit(X_train, Y_train, batch_size=32, epochs=8,
              validation_data=(X_val, Y_val), shuffle=True)

In [ ]:
##### Predict test data #####
Y_pred = model.predict(X_test)
Y_pred = (Y_pred > 0.5)
#print(Y_pred)
#print(Y_test)

Y_pred_labels = np.zeros(len(Y_pred))
Y_test_labels = np.zeros(len(Y_test))
for i in range(len(Y_pred)):
    idx_pred = np.where(Y_pred[i]==True)[0][0]
    Y_pred_labels[i] = idx_pred + 1
    idx_test = np.where(Y_test[i]==1)[0][0]
    Y_test_labels[i] = idx_test + 1

cm = confusion_matrix(Y_test_labels, Y_pred_labels)
print(cm)

#assess classification results
#confusion matrix
plt.rcParams.update({'font.size': 22})
mat = confusion_matrix(Y_test_labels, Y_pred_labels)

f, ax = plt.subplots(1,1,figsize=(12,8))
g = sns.heatmap(mat, annot=True, square=True, fmt="d", cbar=False, xticklabels=[1,2,3,4], yticklabels=[1,2,3,4], ax = ax)
g.set_xlabel('Predicted')
g.set_ylabel('True')
g.set_title('$p_0$ = %.3f, $\kappa$ = %.3f' % (accuracy_score(Y_test_labels, Y_pred_labels), cohen_kappa_score(Y_test_labels, Y_pred_labels)))
plt.show()


epochs = [1, 2, 3, 4, 5, 6, 7, 8]
loss_hist = score_history.history['loss']
accuracy_hist = score_history.history['accuracy']
val_loss_hist = score_history.history['val_loss']
val_accuracy_hist = score_history.history['val_accuracy']

#plot model accuracy
plt.figure(figsize=(12,8))
plt.plot(epochs, accuracy_hist, label='Accuracy')
plt.plot(epochs, val_accuracy_hist, label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='best')
plt.show()

#plot model loss
plt.figure(figsize=(12,8))
plt.plot(epochs, loss_hist, label='Loss')
plt.plot(epochs, val_loss_hist, label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='best')
plt.show()

In [ ]:
##### Get CNN model scores #####

scores = model.evaluate(X_test, Y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [ ]:
#### try with sigmoid activation functions

##### Build a simple CNN model #####
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=X_train.shape[1:]))
model.add(Activation('sigmoid'))
model.add(Flatten())
model.add(Dense(4))
model.add(Activation('sigmoid'))

##### Compile the CNN model #####
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

##### Fit the CNN model #####
score_history = model.fit(X_train, Y_train, batch_size=32, epochs=8,
              validation_data=(X_val, Y_val), shuffle=True)



##### Predict test data #####
Y_pred = model.predict(X_test)
Y_pred = (Y_pred > 0.5)
#print(Y_pred)
#print(Y_test)

Y_pred_labels = np.zeros(len(Y_pred))
Y_test_labels = np.zeros(len(Y_test))
for i in range(len(Y_pred)):
    idx_pred = np.where(Y_pred[i]==True)[0][0]
    Y_pred_labels[i] = idx_pred + 1
    idx_test = np.where(Y_test[i]==1)[0][0]
    Y_test_labels[i] = idx_test + 1


#assess classification results
#confusion matrix
plt.rcParams.update({'font.size': 22})
mat = confusion_matrix(Y_test_labels, Y_pred_labels)

f, ax = plt.subplots(1,1,figsize=(12,8))
g = sns.heatmap(mat, annot=True, square=True, fmt="d", cbar=False, xticklabels=[1,2,3,4], yticklabels=[1,2,3,4], ax = ax)
g.set_xlabel('Predicted')
g.set_ylabel('True')
g.set_title('$p_0$ = %.3f, $\kappa$ = %.3f' % (accuracy_score(Y_test_labels, Y_pred_labels), cohen_kappa_score(Y_test_labels, Y_pred_labels)))
plt.show()

epochs = [1, 2, 3, 4, 5, 6, 7, 8]
loss_hist = score_history.history['loss']
accuracy_hist = score_history.history['accuracy']
val_loss_hist = score_history.history['val_loss']
val_accuracy_hist = score_history.history['val_accuracy']

#plot model accuracy
plt.figure(figsize=(12,8))
plt.plot(epochs, accuracy_hist, label='Accuracy')
plt.plot(epochs, val_accuracy_hist, label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='best')
plt.show()

#plot model loss
plt.figure(figsize=(12,8))
plt.plot(epochs, loss_hist, label='Loss')
plt.plot(epochs, val_loss_hist, label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='best')
plt.show()

##### Get CNN model scores #####
scores = model.evaluate(X_test, Y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [ ]:
#### try with scaled data

##### Normalize the image data #####
X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
X_test = X_test.astype('float32')
X_train_scaled = X_train / 255
X_val_scaled = X_val / 255
X_test_scaled = X_test / 255


##### Build a simple CNN model #####
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(4))
model.add(Activation('softmax'))

##### Compile the CNN model #####
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

##### Fit the CNN model #####
score_history = model.fit(X_train_scaled, Y_train, batch_size=32, epochs=8,
              validation_data=(X_val_scaled, Y_val), shuffle=True)



##### Predict test data #####
Y_pred = model.predict(X_test_scaled)
print(np.shape(Y_pred))
for i in range(len(Y_pred)):
    j = np.argmax(Y_pred[i,:])
    Y_pred[i,j] = 1
print(Y_pred)
#print(Y_test)

Y_pred_labels = np.zeros(len(Y_pred))
Y_test_labels = np.zeros(len(Y_test))
for i in range(len(Y_pred)):
    idx_pred = np.where(Y_pred[i]==1)[0][0]
    Y_pred_labels[i] = idx_pred +1
    idx_test = np.where(Y_test[i]==1)[0][0]
    Y_test_labels[i] = idx_test + 1


#assess classification results
#confusion matrix
plt.rcParams.update({'font.size': 22})
mat = confusion_matrix(Y_test_labels, Y_pred_labels)

f, ax = plt.subplots(1,1,figsize=(12,8))
g = sns.heatmap(mat, annot=True, square=True, fmt="d", cbar=False, xticklabels=[1,2,3,4], yticklabels=[1,2,3,4], ax = ax)
g.set_xlabel('Predicted')
g.set_ylabel('True')
g.set_title('$p_0$ = %.3f, $\kappa$ = %.3f' % (accuracy_score(Y_test_labels, Y_pred_labels), cohen_kappa_score(Y_test_labels, Y_pred_labels)))
plt.show()

epochs = [1, 2, 3, 4, 5, 6, 7, 8]
loss_hist = score_history.history['loss']
accuracy_hist = score_history.history['accuracy']
val_loss_hist = score_history.history['val_loss']
val_accuracy_hist = score_history.history['val_accuracy']

#plot model accuracy
plt.figure(figsize=(12,8))
plt.plot(epochs, accuracy_hist, label='Accuracy')
plt.plot(epochs, val_accuracy_hist, label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='best')
plt.show()

#plot model loss
plt.figure(figsize=(12,8))
plt.plot(epochs, loss_hist, label='Loss')
plt.plot(epochs, val_loss_hist, label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='best')
plt.show()

##### Get CNN model scores #####
scores = model.evaluate(X_test_scaled, Y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])